# Weakly Supervised Recommendation Systems

Experiments steps:
 1. **User's Preferences Model**: Leverage the most *explicit* ratings to build a *rate/rank prediction model*. This is a simple *Explicit Matrix Factorization* model. 
 2. **Generate Weak DataSet**: Use the above model to *predict* for all user/item pairs $(u,i)$ in *implicit feedback dataset* to build a new *weak explicit dataset* $(u, i, r^*)$.
 3. **Evaluate**: Use the intact test split in the most explicit feedback, in order to evaluate the performance of any model.

## Explicit Model Experiments

This section contains all the experiments based on the explicit matrix factorization model.

### Explicit Model

In [1]:
import utils
from spotlight.evaluation import rmse_score

dataset_recommend_train, dataset_recommend_test, dataset_recommend_dev, dataset_read, dataset_shelve = utils.parse_goodreads(
    path='/local/terrier/Collections/Recommendations/Goodreads/goodreads_interactions_children.json.gz')

print('Explicit dataset (TEST) contains %s interactions of %s users and %s items'%(
          format(len(dataset_recommend_test.ratings), ','),
          format(dataset_recommend_test.num_users, ','),
          format(dataset_recommend_test.num_items, ',')))

print('Explicit dataset (VALID) contains %s interactions of %s users and %s items'%(
          format(len(dataset_recommend_dev.ratings), ','),
          format(dataset_recommend_dev.num_users, ','),
          format(dataset_recommend_dev.num_items, ',')))

print('Explicit dataset (TRAIN) contains %s interactions of %s users and %s items'%(
          format(len(dataset_recommend_train.ratings), ','),
          format(dataset_recommend_train.num_users, ','),
          format(dataset_recommend_train.num_items, ',')))

print('Implicit dataset (READ) contains %s interactions of %s users and %s items'%(
          format(len(dataset_read.ratings), ','),
          format(dataset_read.num_users, ','),
          format(dataset_read.num_items, ',')))

print('Implicit dataset (SHELVE) contains %s interactions of %s users and %s items'%(
          format(len(dataset_shelve.ratings), ','),
          format(dataset_shelve.num_users, ','),
          format(dataset_shelve.num_items, ',')))

# train the explicit model based on recommend feedback
model = utils.train_explicit(train_interactions=dataset_recommend_train, 
                             valid_interactions=dataset_recommend_dev,
                             run_name='model_goodreads_children_explicit_rate')

# evaluate the new model
mrr, ndcg, ndcg10, ndcg_5, mmap, success_10, success_5 = utils.evaluate(interactions=dataset_recommend_test,
                                                                        model=model,
                                                                        topk=20)
rmse = rmse_score(model=model, test=dataset_recommend_test)
print('-'*20)
print('RMSE: {:.4f}'.format(rmse))
print('MRR: {:.4f}'.format(mrr))
print('nDCG: {:.4f}'.format(ndcg))
print('nDCG@10: {:.4f}'.format(ndcg10))
print('nDCG@5: {:.4f}'.format(ndcg_5))
print('MAP: {:.4f}'.format(mmap))
print('success@10: {:.4f}'.format(success_10))
print('success@5: {:.4f}'.format(success_5))

Explicit dataset (TEST) contains 550,599 interactions of 144,609 users and 122,321 items
Explicit dataset (VALID) contains 550,600 interactions of 144,609 users and 122,321 items
Explicit dataset (TRAIN) contains 4,404,793 interactions of 144,609 users and 122,321 items
Implicit dataset (READ) contains 5,663,348 interactions of 144,609 users and 122,321 items
Implicit dataset (SHELVE) contains 7,926,668 interactions of 144,609 users and 122,321 items
--------------------
RMSE: 0.4063
MRR: 0.0271
nDCG: 0.0346
nDCG@10: 0.0239
nDCG@5: 0.0139
MAP: 0.0141
success@10: 0.0952
success@5: 0.0442


### Remove all valid/test ratings

In [2]:
test_interact = set()
for (uid, iid) in zip(dataset_recommend_test.user_ids, dataset_recommend_test.item_ids):
    test_interact.add((uid, iid))

for (uid, iid) in zip(dataset_recommend_dev.user_ids, dataset_recommend_dev.item_ids):
    test_interact.add((uid, iid))

# clean implicit dataset from test/dev rating
for idx, (uid, iid, r) in enumerate(zip(dataset_read.user_ids, dataset_read.item_ids, dataset_read.ratings)):
    if (uid, iid) in test_interact:
        dataset_read.ratings[idx] = -1

### Explicit Read Model

Leverage the **explicit rate model** trained at the previous section to annotate **missing values** in the **read** dataset.

In [3]:
# annotate the missing values in the play dataset based on the explicit recommend model
dataset_read = utils.annotate(interactions=dataset_read, 
                              model=model, 
                              run_name='dataset_goodreads_children_read_explicit_annotated')

# train the explicit model based on recommend feedback
model = utils.train_explicit(train_interactions=dataset_read, 
                             valid_interactions=dataset_recommend_dev,
                             run_name='model_goodreads_children_explicit_read')

# evaluate the new model
mrr, ndcg, ndcg10, ndcg_5, mmap, success_10, success_5 = utils.evaluate(interactions=dataset_recommend_test,
                                                                        model=model,
                                                                        topk=20)
rmse = rmse_score(model=model, test=dataset_recommend_test)
print('-'*20)
print('RMSE: {:.4f}'.format(rmse))
print('MRR: {:.4f}'.format(mrr))
print('nDCG: {:.4f}'.format(ndcg))
print('nDCG@10: {:.4f}'.format(ndcg10))
print('nDCG@5: {:.4f}'.format(ndcg_5))
print('MAP: {:.4f}'.format(mmap))
print('success@10: {:.4f}'.format(success_10))
print('success@5: {:.4f}'.format(success_5))

epoch 1 start at:  Tue Apr 23 12:51:05 2019
epoch 1 end at:  Tue Apr 23 12:55:38 2019
RMSE: 0.4099
epoch 2 start at:  Tue Apr 23 12:55:46 2019
epoch 2 end at:  Tue Apr 23 13:00:22 2019
RMSE: 0.4157
--------------------
RMSE: 0.4160
MRR: 0.0833
nDCG: 0.0726
nDCG@10: 0.0641
nDCG@5: 0.0548
MAP: 0.0428
success@10: 0.1862
success@5: 0.1366


## Implicit Model Experiments

This section contains all the experiments based on the implicit matrix factorization model.

### Implicit Model using Negative Sampling

In [3]:
import utils
from spotlight.evaluation import rmse_score

dataset_recommend_train, dataset_recommend_test, dataset_recommend_dev, dataset_read, dataset_shelve = utils.parse_goodreads(
    path='/local/terrier/Collections/Recommendations/Goodreads/goodreads_interactions_children.json.gz')

print('Explicit dataset (TEST) contains %s interactions of %s users and %s items'%(
          format(len(dataset_recommend_test.ratings), ','),
          format(dataset_recommend_test.num_users, ','),
          format(dataset_recommend_test.num_items, ',')))

print('Explicit dataset (VALID) contains %s interactions of %s users and %s items'%(
          format(len(dataset_recommend_dev.ratings), ','),
          format(dataset_recommend_dev.num_users, ','),
          format(dataset_recommend_dev.num_items, ',')))

print('Explicit dataset (TRAIN) contains %s interactions of %s users and %s items'%(
          format(len(dataset_recommend_train.ratings), ','),
          format(dataset_recommend_train.num_users, ','),
          format(dataset_recommend_train.num_items, ',')))

print('Implicit dataset (READ) contains %s interactions of %s users and %s items'%(
          format(len(dataset_read.ratings), ','),
          format(dataset_read.num_users, ','),
          format(dataset_read.num_items, ',')))

print('Implicit dataset (SHELVE) contains %s interactions of %s users and %s items'%(
          format(len(dataset_shelve.ratings), ','),
          format(dataset_shelve.num_users, ','),
          format(dataset_shelve.num_items, ',')))

# train the explicit model based on recommend feedback
model = utils.train_implicit_negative_sampling(train_interactions=dataset_read, 
                                               valid_interactions=dataset_recommend_dev,
                                               run_name='model_goodreads_children_implicit')

# evaluate the new model
mrr, ndcg, ndcg10, ndcg_5, mmap, success_10, success_5 = utils.evaluate(interactions=dataset_recommend_test,
                                                                        model=model,
                                                                        topk=20)
rmse = rmse_score(model=model, test=dataset_recommend_test)
print('-'*20)
print('RMSE: {:.4f}'.format(rmse))
print('MRR: {:.4f}'.format(mrr))
print('nDCG: {:.4f}'.format(ndcg))
print('nDCG@10: {:.4f}'.format(ndcg10))
print('nDCG@5: {:.4f}'.format(ndcg_5))
print('MAP: {:.4f}'.format(mmap))
print('success@10: {:.4f}'.format(success_10))
print('success@5: {:.4f}'.format(success_5))

Explicit dataset (TEST) contains 550,599 interactions of 144,609 users and 122,321 items
Explicit dataset (VALID) contains 550,600 interactions of 144,609 users and 122,321 items
Explicit dataset (TRAIN) contains 4,404,793 interactions of 144,609 users and 122,321 items
Implicit dataset (READ) contains 5,663,348 interactions of 144,609 users and 122,321 items
Implicit dataset (SHELVE) contains 7,926,668 interactions of 144,609 users and 122,321 items
epoch 1 start at:  Sat Apr 20 13:30:52 2019
epoch 1 end at:  Sat Apr 20 13:36:11 2019
MRR: 0.1126
epoch 2 start at:  Sat Apr 20 14:28:33 2019
epoch 2 end at:  Sat Apr 20 14:32:54 2019
MRR: 0.1129
epoch 3 start at:  Sat Apr 20 15:19:30 2019
epoch 3 end at:  Sat Apr 20 15:23:50 2019
MRR: 0.1131
epoch 4 start at:  Sat Apr 20 16:10:24 2019
epoch 4 end at:  Sat Apr 20 16:14:44 2019
MRR: 0.1129
--------------------
RMSE: 4.0915
MRR: 0.1127
nDCG: 0.1072
nDCG@10: 0.0875
nDCG@5: 0.0694
MAP: 0.0606
success@10: 0.2578
success@5: 0.1682


## Popularity Model

In [4]:
import utils
from popularity import PopularityModel
from spotlight.evaluation import rmse_score

dataset_recommend_train, dataset_recommend_test, dataset_recommend_dev, dataset_read, dataset_shelve = utils.parse_goodreads(
    path='/local/terrier/Collections/Recommendations/Goodreads/goodreads_interactions_children.json.gz')

print('Explicit dataset (TEST) contains %s interactions of %s users and %s items'%(
          format(len(dataset_recommend_test.ratings), ','),
          format(dataset_recommend_test.num_users, ','),
          format(dataset_recommend_test.num_items, ',')))

print('Explicit dataset (VALID) contains %s interactions of %s users and %s items'%(
          format(len(dataset_recommend_dev.ratings), ','),
          format(dataset_recommend_dev.num_users, ','),
          format(dataset_recommend_dev.num_items, ',')))

print('Explicit dataset (TRAIN) contains %s interactions of %s users and %s items'%(
          format(len(dataset_recommend_train.ratings), ','),
          format(dataset_recommend_train.num_users, ','),
          format(dataset_recommend_train.num_items, ',')))

print('Implicit dataset (READ) contains %s interactions of %s users and %s items'%(
          format(len(dataset_read.ratings), ','),
          format(dataset_read.num_users, ','),
          format(dataset_read.num_items, ',')))

print('Implicit dataset (SHELVE) contains %s interactions of %s users and %s items'%(
          format(len(dataset_shelve.ratings), ','),
          format(dataset_shelve.num_users, ','),
          format(dataset_shelve.num_items, ',')))

# train the explicit model based on recommend feedback
model = PopularityModel()
print('fit the model')
model.fit(interactions=dataset_recommend_train)

# evaluate the new model
print('evaluate the model')
mrr, ndcg, ndcg10, ndcg_5, mmap, success_10, success_5 = utils.evaluate(interactions=dataset_recommend_test,
                                                                        model=model,
                                                                        topk=20)
# rmse = rmse_score(model=model, test=dataset_recommend_test, batch_size=512)
# print('-'*20)
# print('RMSE: {:.4f}'.format(rmse))
print('MRR: {:.4f}'.format(mrr))
print('nDCG: {:.4f}'.format(ndcg))
print('nDCG@10: {:.4f}'.format(ndcg10))
print('nDCG@5: {:.4f}'.format(ndcg_5))
print('MAP: {:.4f}'.format(mmap))
print('success@10: {:.4f}'.format(success_10))
print('success@5: {:.4f}'.format(success_5))

Explicit dataset (TEST) contains 550,599 interactions of 144,609 users and 122,321 items
Explicit dataset (VALID) contains 550,600 interactions of 144,609 users and 122,321 items
Explicit dataset (TRAIN) contains 4,404,793 interactions of 144,609 users and 122,321 items
Implicit dataset (READ) contains 5,663,348 interactions of 144,609 users and 122,321 items
Implicit dataset (SHELVE) contains 7,926,668 interactions of 144,609 users and 122,321 items
fit the model
evaluate the model
MRR: 0.1122
nDCG: 0.1067
nDCG@10: 0.0872
nDCG@5: 0.0691
MAP: 0.0605
success@10: 0.2560
success@5: 0.1662
